In [1]:
! nvidia-smi

Tue Aug 18 13:19:41 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Word Tokenizer exercise##

In this exercise, you are going to build a set of deep learning models on a (sort of) real world task using Tensorflow and Keras. Tensorflow is a deep learning framwork developed by Google, and Keras is a frontend library built on top of Tensorflow (or Theano, CNTK) to provide an easier way to use standard layers and networks.

To complete this exercise, you will need to build deep learning models for word tokenization in Thai (แบ่งเว้นวรรคภาษาไทย) using NECTEC's BEST corpus. You will build one model for each of the following type:
- Fully Connected (Feedforward) Neural Network
- One-Dimentional Convolution Neural Network (1D-CNN)
- Recurrent Neural Network with Gated Recurrent Unit (GRU)

and one more model of your choice to achieve the highest score possible.

We provide the code for data cleaning and some starter code for keras in this notebook but feel free to modify those parts to suit your needs. You can also complete this exercise using only Tensorflow (without using Keras). Feel free to use additional libraries (e.g. scikit-learn) as long as you have a model for each type mentioned above.

This notebook assumes you have already installed Tensorflow and Keras with python3 and had GPU enabled. If you run this exercise on GCloud using the provided disk image you are all set.

As a reminder,

### Don't forget to shut down your instance on Gcloud when you are not using it ###

In [2]:
# Run setup code
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gc

from IPython.core.debugger import set_trace

%matplotlib inline
%load_ext tensorboard

In [3]:
# ! pip install gdown -q

In [4]:
# ! gdown https://drive.google.com/uc?id=1iodAqVNWEkiJgH8cWkccsLi_tqoFcMrV
# ! tar -xf corpora.tar.gz

In [5]:
# # Prepare data
# # You don't need to run the following code as we already did it for you to give everyone the same dataset
# import cattern.data_utils
# cattern.data_utils.generate_best_dataset(os.getcwd()+'/data', create_val=True)

For simplicity, we are going to build a word tokenization model which is a binary classification model trying to predict whether a character is the begining of the word or not (if it is, then there is a space in front of it) and without using any knowledge about type of character (vowel, number, English character etc.).

For example,

'แมวดำน่ารักมาก' -> 'แมว ดำ น่า รัก มาก'

will have these true labels:

[(แ,1), (ม,0), (ว,0) (ด,1), ( ำ,0), (น,1), (-่,0), (า,0), (ร,1), (-ั,0), (ก,0), (ม,1), (า,0), (ก,0)]

In this task, we will use only main character you are trying to predict  and the characters that surround it (the context) as features. However, you can imagine that a more complex model will try to include more knowledge about each character into the model. You can do that too if you feel like it.

In [6]:
# Create a character map
CHARS = [
  '\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+',
  ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8',
  '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E',
  'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R',
  'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_',
  'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
  'n', 'o', 'other', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y',
  'z', '}', '~', 'ก', 'ข', 'ฃ', 'ค', 'ฅ', 'ฆ', 'ง', 'จ', 'ฉ', 'ช',
  'ซ', 'ฌ', 'ญ', 'ฎ', 'ฏ', 'ฐ', 'ฑ', 'ฒ', 'ณ', 'ด', 'ต', 'ถ', 'ท',
  'ธ', 'น', 'บ', 'ป', 'ผ', 'ฝ', 'พ', 'ฟ', 'ภ', 'ม', 'ย', 'ร', 'ฤ',
  'ล', 'ว', 'ศ', 'ษ', 'ส', 'ห', 'ฬ', 'อ', 'ฮ', 'ฯ', 'ะ', 'ั', 'า',
  'ำ', 'ิ', 'ี', 'ึ', 'ื', 'ุ', 'ู', 'ฺ', 'เ', 'แ', 'โ', 'ใ', 'ไ',
  'ๅ', 'ๆ', '็', '่', '้', '๊', '๋', '์', 'ํ', '๐', '๑', '๒', '๓',
  '๔', '๕', '๖', '๗', '๘', '๙', '‘', '’', '\ufeff'
]
CHARS_MAP = {v: k for k, v in enumerate(CHARS)}

In [7]:
# CHARS[80], len(CHARS_MAP), CHARS_MAP

In [8]:
def create_n_gram_df(df, n_pad):
  """
  Given an input dataframe, create a feature dataframe of shifted characters
  Input:
  df: timeseries of size (N)
  n_pad: the number of context. For a given character at position [idx],
    character at position [idx-n_pad/2 : idx+n_pad/2] will be used 
    as features for that character.
  
  Output:
  dataframe of size (N * n_pad) which each row contains the character, 
    n_pad_2 characters to the left, and n_pad_2 characters to the right
    of that character.
  """
  n_pad_2 = int((n_pad - 1)/2)
  for i in range(n_pad_2):
      df['char-{}'.format(i+1)] = df['char'].shift(i + 1)
      df['char{}'.format(i+1)] = df['char'].shift(-i - 1)
  return df[n_pad_2: -n_pad_2]

In [9]:
def prepare_feature(best_processed_path, option='train'):
  """
  Transform the path to a directory containing processed files 
  into a feature matrix and output array
  Input:
  best_processed_path: str, path to a processed version of the BEST dataset
  option: str, 'train' or 'test'
  """
  # we use padding equals 21 here to consider 10 characters to the left
  # and 10 characters to the right as features for the character in the middle
  n_pad = 21
  n_pad_2 = int((n_pad - 1)/2)
  pad = [{'char': ' ', 'target': True}]
  df_pad = pd.DataFrame(pad * n_pad_2)
#   print(df_pad)

  dfs = []
  # article types in BEST corpus
#   article_types = ['novel', 'news']               # use smaller dataset to train faster
  article_types = ['article', 'encyclopedia', 'news', 'novel']
  for article_type in article_types:
      dfs.append(pd.read_csv(os.path.join(best_processed_path, option, 'df_best_{}_{}.csv'.format(article_type, option))))

#   set_trace()
  df = pd.concat(dfs)
  # pad with empty string feature
  df = pd.concat((df_pad, df, df_pad))

  # map characters to numbers, use 'other' if not in the predefined character set.
  df['char'] = df['char'].map(lambda x: CHARS_MAP.get(x, 80))

  # Use nearby characters as features
  df_with_context = create_n_gram_df(df, n_pad=n_pad)

  char_row = ['char' + str(i + 1) for i in range(n_pad_2)] + \
             ['char-' + str(i + 1) for i in range(n_pad_2)] + ['char']

  # convert pandas dataframe to numpy array to feed to the model
  x_char = df_with_context[char_row]#.values()
  y = df_with_context['target'].astype(int)#.values()

  return x_char, y

Before running the following commands, we must inform you that our data is quite large and loading the whole dataset at once will **use a lot of memory (~6 GB after processing and up to ~12GB while processing)**. We expect you to be running this on Google Cloud so that you will not run into this problem. But, if, for any reason, you have to run this on your PC or machine with not enough memory, you might need to write a data generator to process a few entries at a time then feed it to the model while training.

For keras, you can use [fit_generator](https://keras.io/getting-started/faq/#how-can-i-use-keras-with-datasets-that-dont-fit-in-memory) to cope with that.

In [10]:
# Path to the preprocessed data
best_processed_path = 'corpora/BEST'

In [11]:
# prepare_feature??

In [12]:
# Load preprocessed BEST corpus
x_train_char, y_train = prepare_feature(best_processed_path, option='train')
x_val_char, y_val = prepare_feature(best_processed_path, option='val')
x_test_char, y_test = prepare_feature(best_processed_path, option='test')

# As a sanity check, we print out the size of the training, val, and test data.
print('Training data shape: ', x_train_char.shape)
print('Training data labels shape: ', y_train.shape)
print('Validation data shape: ', x_val_char.shape)
print('Validation data labels shape: ', y_val.shape)
print('Test data shape: ', x_test_char.shape)
print('Test data labels shape: ', y_test.shape)

Training data shape:  (16461637, 21)
Training data labels shape:  (16461637,)
Validation data shape:  (2035694, 21)
Validation data labels shape:  (2035694,)
Test data shape:  (2271932, 21)
Test data labels shape:  (2271932,)


In [13]:
# Print some entry from the data to make sure it is the same as what you think.
print('First 3 features: ')
x_train_char[:3]

First 3 features: 


,char1,char2,char3,char4,char5,char6,char7,char8,char9,char10,char-1,char-2,char-3,char-4,char-5,char-6,char-7,char-8,char-9,char-10,char
0,112.0,140.0,114.0,148.0,130.0,142.0,94.0,142.0,128.0,128.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,97
1,140.0,114.0,148.0,130.0,142.0,94.0,142.0,128.0,128.0,141.0,97.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,112
2,114.0,148.0,130.0,142.0,94.0,142.0,128.0,128.0,141.0,109.0,112.0,97.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,140


In [14]:
print('First 30 class labels')
y_train[:30]

First 30 class labels


0     1
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    1
21    0
22    0
23    1
24    0
25    0
26    0
27    1
28    0
29    0
Name: target, dtype: int64

In [15]:
x_train_char.head(15)

,char1,char2,char3,char4,char5,char6,char7,char8,char9,char10,char-1,char-2,char-3,char-4,char-5,char-6,char-7,char-8,char-9,char-10,char
0,112.0,140.0,114.0,148.0,130.0,142.0,94.0,142.0,128.0,128.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,97
1,140.0,114.0,148.0,130.0,142.0,94.0,142.0,128.0,128.0,141.0,97.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,112
2,114.0,148.0,130.0,142.0,94.0,142.0,128.0,128.0,141.0,109.0,112.0,97.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,140
3,148.0,130.0,142.0,94.0,142.0,128.0,128.0,141.0,109.0,117.0,140.0,112.0,97.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,114
4,130.0,142.0,94.0,142.0,128.0,128.0,141.0,109.0,117.0,128.0,114.0,140.0,112.0,97.0,1.0,1.0,1.0,1.0,1.0,1.0,148
5,142.0,94.0,142.0,128.0,128.0,141.0,109.0,117.0,128.0,128.0,148.0,114.0,140.0,112.0,97.0,1.0,1.0,1.0,1.0,1.0,130
6,94.0,142.0,128.0,128.0,141.0,109.0,117.0,128.0,128.0,126.0,130.0,148.0,114.0,140.0,112.0,97.0,1.0,1.0,1.0,1.0,142
7,142.0,128.0,128.0,141.0,109.0,117.0,128.0,128.0,126.0,118.0,142.0,130.0,148.0,114.0,140.0,112.0,97.0,1.0,1.0,1.0,94
8,128.0,128.0,141.0,109.0,117.0,128.0,128.0,126.0,118.0,149.0,94.0,142.0,130.0,148.0,114.0,140.0,112.0,97.0,1.0,1.0,142
9,128.0,141.0,109.0,117.0,128.0,128.0,126.0,118.0,149.0,106.0,142.0,94.0,142.0,130.0,148.0,114.0,140.0,112.0,97.0,1.0,128


In [16]:
#print char of feature 1
char = np.array(CHARS)

#A function for displaying our features in text
def print_features(tfeature,label,index):
    # set_trace()
    feature = np.array(tfeature.iloc[index],dtype=int).reshape(21,1)
    #Convert to string
    char_list = char[feature]
    # print(char_list)
    left = ''.join(reversed(char_list[10:20].reshape(10))).replace(" ", "")
    center = ''.join(char_list[20])
    right =  ''.join(char_list[0:10].reshape(10)).replace(" ", "")
    word = ''.join([left,' ',center,' ',right])
    print(center + ': ' + word + "\t pred = "+str(label.iloc[index]))

for ind in range(1000,1030):
    print_features(x_train_char,y_train,ind)

บ: ำเร็จสำหรั บ การปฏิวัติ	 pred = 0
ก: เร็จสำหรับ ก ารปฏิวัติห	 pred = 1
า: ร็จสำหรับก า รปฏิวัติหร	 pred = 0
ร: ็จสำหรับกา ร ปฏิวัติหรื	 pred = 0
ป: จสำหรับการ ป ฏิวัติหรือ	 pred = 1
ฏ: สำหรับการป ฏ ิวัติหรือร	 pred = 0
ิ: ำหรับการปฏ ิ วัติหรือรั	 pred = 0
ว: หรับการปฏิ ว ัติหรือรัฐ	 pred = 0
ั: รับการปฏิว ั ติหรือรัฐป	 pred = 0
ต: ับการปฏิวั ต ิหรือรัฐปร	 pred = 0
ิ: บการปฏิวัต ิ หรือรัฐประ	 pred = 0
ห: การปฏิวัติ ห รือรัฐประห	 pred = 1
ร: ารปฏิวัติห ร ือรัฐประหา	 pred = 0
ื: รปฏิวัติหร ื อรัฐประหาร	 pred = 0
อ: ปฏิวัติหรื อ รัฐประหารท	 pred = 0
ร: ฏิวัติหรือ ร ัฐประหารที	 pred = 1
ั: ิวัติหรือร ั ฐประหารที่	 pred = 0
ฐ: วัติหรือรั ฐ ประหารที่จ	 pred = 0
ป: ัติหรือรัฐ ป ระหารที่จะ	 pred = 0
ร: ติหรือรัฐป ร ะหารที่จะต	 pred = 0
ะ: ิหรือรัฐปร ะ หารที่จะต้	 pred = 0
ห: หรือรัฐประ ห ารที่จะต้อ	 pred = 0
า: รือรัฐประห า รที่จะต้อง	 pred = 0
ร: ือรัฐประหา ร ที่จะต้องใ	 pred = 0
ท: อรัฐประหาร ท ี่จะต้องให	 pred = 1
ี: รัฐประหารท ี ่จะต้องให้	 pred = 0
่: ัฐประหารที ่ จะต้องให้ร	 pred = 0
จ

Now, you are going to define the model to be used as your classifier. If you are using Keras, please follow the guideline we provide below. We encourage you to use the Keras Funtional API instead of the Sequential model as it is easier to create complex models (and to read your code). You can find more about Keras Functional API on Keras [documentation](https://keras.io/getting-started/functional-api-guide/).

You need to create a function that return a model you want and write code that invoke _**model.fit**_ to train your model. For example, your function ***get_nn()*** might looks like this:

```python
def get_nn():
  input1 = Input(...)
  ...
  # Create your wonderful model.
  ...
  out = ...
  model = Model(inputs=input1, outputs=out)
  model.compile(...)
return model
```

Also, beware that complex model requires more time to train and your dataset is already quite large. We tested it with a simple 1-hidden-layered feedforward nueral network and it used ~10 mins to train 1 epoch. It got more than 95% accuracy on validation set after the first epoch, so you should aim for a model with accuracy around 96-98%.

# Three-Layer Feedforward Neural Networks

Below, we provide you the code for creating a 3-layer fully connected neural network in keras. This will also serve as the baseline for your other models. Run the code below while making sure you understand what you are doing. Then, report the results.

In [17]:
from keras.models import Model
from keras.layers import Input, Dense, Embedding, TimeDistributed, Conv1D, Flatten, GRU, Dropout
from keras.optimizers import Adam

def get_feedforward_nn():
  input1 = Input(shape=(21,))
  x = Dense(100, activation='relu')(input1)
  x = Dense(100, activation='relu')(x)
  x = Dense(100, activation='relu')(x)
  out = Dense(1, activation='sigmoid')(x)

  model = Model(inputs=input1, outputs=out)
  model.compile(optimizer=Adam(),
                loss='binary_crossentropy',
                metrics=['acc'])
  return model

In [18]:
from keras.callbacks import ModelCheckpoint, TensorBoard

# This is called to clear the original model session in order to use TensorBoard
from keras import backend as K
K.clear_session()

# Path to save model parameters
weight_path_feedforward_nn='/data/model_weight_feedforward_nn.h5'

# Training callbacks list. TensorBoard() write logs for tensorboard GUI. 
# ModelCheckpoint() writes the resulting model.
# Note that writing to disk takes time (longer than model training time). 
# For other sections, you might not writing any files to disk 
# or write only the graph for TensorBoard.

tboard_path = '/data/Graph/ff'
callbacks_list_feedforward_nn = [
        TensorBoard(log_dir=tboard_path, histogram_freq=1, write_graph=True, write_grads=True),
        ModelCheckpoint(
            weight_path_feedforward_nn,
            save_best_only=True,
            save_weights_only=True,
            monitor='val_loss',
            mode='min',
            verbose=1
        )
  ]

verbose = 1
model_feedforward_nn = get_feedforward_nn()

In [19]:
print('start training')
train_params = [(3, 512)]
for (epochs, batch_size) in train_params:
  print("train with {} epochs and {} batch size".format(epochs, batch_size))
  model_feedforward_nn.fit(x_train_char, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose,
                        #    callbacks=callbacks_list_feedforward_nn,       # comment for faster training
                           validation_data=(x_val_char, y_val))

start training
train with 3 epochs and 512 batch size
Epoch 1/3
32152/32152 [==============================] - 99s 3ms/step - loss: 0.3427 - acc: 0.8511 - val_loss: 0.2924 - val_acc: 0.8749
Epoch 2/3
32152/32152 [==============================] - 98s 3ms/step - loss: 0.2692 - acc: 0.8864 - val_loss: 0.2638 - val_acc: 0.8892
Epoch 3/3
32152/32152 [==============================] - 97s 3ms/step - loss: 0.2463 - acc: 0.8972 - val_loss: 0.2547 - val_acc: 0.8897


In [21]:
################################################################################
# In case you want to close the session, you can re-load the model by          #
################################################################################
# weight_path_feedforward_nn='/data/model_weight_feedforward_nn.h5'
# model_feedforward_nn = get_feedforward_nn()
# model_feedforward_nn.load_weights(weight_path_feedforward_nn)
# model_feedforward_nn._make_predict_function()
# model_feedforward_nn.summary()

In [19]:
from sklearn.metrics import f1_score,precision_score,recall_score

################################################################################
# Write a function to evaluate your model. Your function must make prediction  #
# using the input model and return f-score, precision, and recall of the model.#
# You can make predictions by calling model.predict().                         #
################################################################################
def evaluate(x_test, y_test, model):
  """
  Evaluate model on the splitted 10 percent testing set.
  """
  y_pred = model.predict(x_test)

  #map probability to class
  prob_to_class = lambda p: 1 if p[0]>=0.5 else 0
  y_pred = np.apply_along_axis(prob_to_class,1,y_pred)
    
  f1score = f1_score(y_test,y_pred)
  precision = precision_score(y_test,y_pred)
  recall = recall_score(y_test,y_pred)
  return f1score, precision, recall

In [23]:
evaluate(x_test_char, y_test, model_feedforward_nn)

(0.7795816217738168, 0.9015546513026634, 0.686679461965579)

# Debugging

In order to understand what is going on in your model and where the error is, you should try looking at the inputs your model made wrong predictions.

In this task, write a function to print the characters on test data that got wrong prediction along with its context of size 10 (from [x-10] to [x+10]). Examine a fews of those and write your assumption on where the model got wrong prediction.

In [23]:
# TODO#1
# Write code to show a few of the errors the models made.
y_pred = model_feedforward_nn.predict(x_test_char)
len(y_pred)

2271932

In [24]:
model_feedforward_nn = None
gc.collect()

7908

In [25]:
prob_to_class = lambda p: 1 if p[0]>=0.5 else 0
y_pred = np.apply_along_axis(prob_to_class,1,y_pred)

res = y_test[:100] == y_pred[:100]
res

0     True
1     True
2     True
3     True
4     True
      ... 
95    True
96    True
97    True
98    True
99    True
Name: target, Length: 100, dtype: bool

In [26]:
not_match_idx = [i for i, x in enumerate(res) if not x]
not_match_idx

[6, 32, 40, 45, 50, 51]

In [27]:
y_test.iloc[not_match_idx]

6     1
32    1
40    1
45    1
50    1
51    0
Name: target, dtype: int64

In [28]:
y_pred_df = pd.Series(y_pred)
for ind in range((len(not_match_idx))):
    print_features(x_test_char.iloc[not_match_idx], y_pred_df.iloc[not_match_idx],ind)

ก: ปฏิรูป ก ารศึกษา:	 pred = 0
ท: งทางกระบวน ท ัศน์และบริ	 pred = 0
บ: วนทัศน์และ บ ริบทสังคมไ	 pred = 0
ส: น์และบริบท ส ังคมไทยThe	 pred = 0
ไ: บริบทสังคม ไ ทยTheRefo	 pred = 0
ท: ริบทสังคมไ ท ยTheRefor	 pred = 1


# Write your answer here

**Your answer**: TODO#2

# Tensorboard #
The code provided also have Tensorboard (a visualization tool that comes with Tensorflow). Note the part that calls it `TensorBoard(log_dir='./Graph/' + graph_name, histogram_freq=1, write_graph=True, write_grads=True)`. This tells Tensorflow to write extra outputs to the `log_dir` which can then be used for visualization.

To start tensorboard do
```
tensorboard --logdir=/full_path_to_your_logs
```
from the commandline. This will launch tensorboard, you will be able to access it from a web browser by pointint the url to `<instance-ip>:6006`. You will need to enable additional firewall rules in Gcloud for this.

In Tensorboard, you will be able to debug your computation graph which can be hard to keep track in code. This is might seem trivial in Keras, but it is very helpful for Tensorflow. You can see a visualization of the computation graph at the `GRAPH` tab. If you see multiple dense layers (more than 4), this is caused by running the code several times without deleting the log dir. Delete the log dir and re-run the code.

Next, let's look at the scalars tab, we can see the loss and accuracy on the training and validation set as they change over each epoch. This can be useful to detect overfitting.

Another useful tab is the histograms tab. This plot histograms of the weights, biases, and outputs of each layer. The depth of the histograms show the change over epochs. We can see how the histograms of weights change over the training peroid. This can be used to debug vanishing gradients or getting stuck in local minimas.

There are other useful tabs in Tensorboard, you can read about them in the Keras [documentation](https://keras.io/callbacks/#tensorboard) for tensorboard.

In [29]:
# ## clear tensorboard logs
# ! rm -rf $tboard_path

In [30]:
# %tensorboard --logdir=$tboard_path

# Tensorboard observation

Write your own interpretation of the logs from this example. A simple sentence or two for each tab is sufficient.

**Your answer**: TODO#3

# Dropout

You might notice that the 3-layered feedforward does not use dropout at all. Now, try adding dropout to the model, run, and report the result again.

In [24]:
# TODO#4
# Write a function that return feedforward model with dropout

from keras.layers import Dropout

def get_nn_with_dropout():
  input1 = Input(shape=(21,))
  x = Dense(100, activation='relu')(input1)
  x = Dense(100, activation='relu')(x)
  x = Dense(100, activation='relu')(x)
  x = Dropout(0.5)(x)
  out = Dense(1, activation='sigmoid')(x)

  model = Model(inputs=input1, outputs=out)
  model.compile(optimizer=Adam(),
                loss='binary_crossentropy',
                metrics=['acc'])
  return model

In [25]:
# Train your model
print('start training')
verbose = 1
model_nn_with_dropout = get_nn_with_dropout()
# TODO#5
# Complete the code to train your model with dropout
################################################################################
#                            WRITE YOUR CODE BELOW                             #
################################################################################
train_params = [(3, 512)]
for (epochs, batch_size) in train_params:
  print("train with {} epochs and {} batch size".format(epochs, batch_size))
  model_nn_with_dropout.fit(x_train_char, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose,
                        #    callbacks=callbacks_list_feedforward_nn,       # comment for faster training
                           validation_data=(x_val_char, y_val))

start training
train with 3 epochs and 512 batch size
Epoch 1/3
32152/32152 [==============================] - 106s 3ms/step - loss: 0.3811 - acc: 0.8338 - val_loss: 0.3158 - val_acc: 0.8614
Epoch 2/3
32152/32152 [==============================] - 102s 3ms/step - loss: 0.3132 - acc: 0.8644 - val_loss: 0.2861 - val_acc: 0.8747
Epoch 3/3
32152/32152 [==============================] - 102s 3ms/step - loss: 0.2918 - acc: 0.8757 - val_loss: 0.2637 - val_acc: 0.8888


In [26]:
evaluate(x_test_char, y_test, model_nn_with_dropout)

(0.7905922892045296, 0.8552695429680607, 0.7350093595242044)

In [27]:
model_nn_with_dropout = None
gc.collect()

8213

# Convolution Neural Networks

Now, you are going to implement you own 1d-convolution neural networks with the following structure:
input -> embedding layer (size 32) -> 1D-convolution layer (100 filters of size 5, strides of 1) -> TimeDistributed (Dense size 5) -> fully-connected layer (size 100) -> output.

These parameters are simple guidelines to save your time. You can play with them in the final section.

The results should be better than the feedforward model.

Embedding layers turn the input from a one-hot vector into better representations via some feature transform (a simple matrix multiply in this case). TimeDistributed is Keras' way of specifying that the layer of the network should be distributed along time (the first dimension) as shown in the picture below.

<img src="https://raw.githubusercontent.com/ekapolc/nlp_course/master/HW1/images/configuration.png">

Note you need to flatten() before the final fully connected layer because of dimension mis-match.

Do consult keras documentation on how to use [embedding layers](https://keras.io/layers/embeddings/) and [1D-cnn](https://keras.io/layers/convolutional/).


In [28]:
################################################################################
# TODO#6:                                                                      #
# Write a function that returns keras convolution nueral network model.        #
# You can choose any normalization methods, activation function, as well as    #
# any hyperparameter the way you want. Your goal is to predict a score         #
# between [0,1] for each input whether it is the beginning of the word or not. #
#                                                                              #
# Hint: You should read keras documentation to see the list of available       #
# layers and options you can use.                                              #
################################################################################

def get_conv1d_nn():
  input1 = Input(shape=(21,))

  x = Embedding(len(CHARS), 32)(input1)  
  x = Conv1D(100, 12, activation='relu')(x)
  x = TimeDistributed(Dense(5, activation='relu'))(x)
  x = Flatten()(x)
  x = Dense(100, activation='relu')(x)
  x = Dropout(0.5)(x)
  out = Dense(1, activation='sigmoid')(x)

  model = Model(inputs=input1, outputs=out)
  model.compile(optimizer=Adam(),
                loss='binary_crossentropy',
                metrics=['acc'])
  return model

In [29]:
################################################################################
# TODO#7:                                                                      #
# Write code that call model.fit, or model.fit_generator if you have data      #
# generator, to train you models. Make sure you have validation_data as an     # 
# argument and use verbose=2 to generate one log line per epoch. Select your   #
# batch size carefully as it will affect your model's ability to converge and  #
# time needed for one epoch.                                                   #
################################################################################
print('start training conv1d')
model_conv1d_nn = get_conv1d_nn()
################################################################################
#                            WRITE YOUR CODE BELOW                             #
################################################################################
train_params = [(3, 512)]
for (epochs, batch_size) in train_params:
  print("train with {} epochs and {} batch size".format(epochs, batch_size))
  model_conv1d_nn.fit(x_train_char, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose,
                        #    callbacks=callbacks_list_feedforward_nn,
                           validation_data=(x_val_char, y_val))

start training conv1d
train with 3 epochs and 512 batch size
Epoch 1/3
32152/32152 [==============================] - 212s 7ms/step - loss: 0.0804 - acc: 0.9703 - val_loss: 0.0642 - val_acc: 0.9767
Epoch 2/3
32152/32152 [==============================] - 189s 6ms/step - loss: 0.0573 - acc: 0.9797 - val_loss: 0.0571 - val_acc: 0.9797
Epoch 3/3
32152/32152 [==============================] - 188s 6ms/step - loss: 0.0528 - acc: 0.9814 - val_loss: 0.0550 - val_acc: 0.9806


In [30]:
evaluate(x_test_char, y_test, model_conv1d_nn)

(0.9680909519483236, 0.9568038089419417, 0.9796475763966227)

In [31]:
model_conv1d_nn = None
gc.collect()

10484

# GRU

Implement your GRU model with the following structure: input -> embedding layer (size 32) -> GRU layer (size 32) -> fully-connected layer (size 100) -> output.


These parameters are simple guidelines to save your time. You can play with them in the final section.

The result should be better than the feedforward model and at least on par with your CNN model.

Do consult keras documentation on how to use [embedding layers](https://keras.io/layers/embeddings/) and [GRUs](https://keras.io/layers/recurrent/).


In [19]:
################################################################################
# TODO#8                                                                       #
# Write a function that returns keras GRU network moded. You can choose any    #
# normalization methods, activation function, as well as any hyperparameter    #
# the way you want. Your goal is to predict a score between [0,1] for each     #
# input whether it is the beginning of the word or not.                        #
#                                                                              #
# Hint: You should read keras documentation to see the list of available       #
# layers and options you can use.                                              #
################################################################################

def get_gru():
  input1 = Input(shape=(21,))

  x = Embedding(len(CHARS), 32)(input1)  
  x = GRU(32)(x)
  x = Dense(100, activation='relu')(x)
  x = Dropout(0.5)(x)
  out = Dense(1, activation='sigmoid')(x)

  model = Model(inputs=input1, outputs=out)
  model.compile(optimizer=Adam(),
                loss='binary_crossentropy',
                metrics=['acc'])
  return model

In [20]:
################################################################################
# TODO#9                                                                       #
# Write code that call model.fit, or model.fit_generator if you have data      #
# generator, to train you models. Make sure you have validation_data as an     # 
# argument and use verbose=2 to generate one log line per epoch. Select your   #
# batch size carefully as it will affect your model's ability to converge and  #
# time needed for one epoch.                                                   #
################################################################################
print('start training gru')
model_gru = get_gru()
################################################################################
#                            WRITE YOUR CODE BELOW                             #
################################################################################
train_params = [(3, 512)]
for (epochs, batch_size) in train_params:
  print("train with {} epochs and {} batch size".format(epochs, batch_size))
  model_gru.fit(x_train_char, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose,
                        #    callbacks=callbacks_list_feedforward_nn,
                           validation_data=(x_val_char, y_val))

start training gru
train with 3 epochs and 512 batch size
Epoch 1/3
32152/32152 [==============================] - 230s 7ms/step - loss: 0.1402 - acc: 0.9420 - val_loss: 0.0902 - val_acc: 0.9651
Epoch 2/3
32152/32152 [==============================] - 216s 7ms/step - loss: 0.0814 - acc: 0.9692 - val_loss: 0.0786 - val_acc: 0.9705
Epoch 3/3
32152/32152 [==============================] - 213s 7ms/step - loss: 0.0730 - acc: 0.9730 - val_loss: 0.0723 - val_acc: 0.9732


In [24]:
evaluate(x_test_char, y_test, model_gru)

(0.9560793829083485, 0.9392820290840255, 0.9734884565868145)

In [25]:
model_gru = None
gc.collect()

33896

# Final Section
# Keras playground

Now, train the best model you can do for this task. You can use any model structure and function available. Remember that trainig time increases with the complexity of the model. You might find TensorBoard helpful in tuning of complicated models.

Your model should be better than your CNN or GRU model in the previous sections.

Some ideas to try
1. Tune the parameters
2. Bi-directional GRU
3. CNN-GRU model

In [20]:
################################################################################
# TODO#10                                                                      #
# Write a function that returns keras your best model. You can use anything    #
# you want. The goal here is to create the best model you can think of.        #
# Your model should get f-score more than 97% from calling evaluate().         #
#                                                                              #
# Hint: You should read keras documentation to see the list of available       #
# layers and options you can use.                                              #
################################################################################

def get_my_best_model():
  input1 = Input(shape=(21,))

  x = Embedding(len(CHARS), 32)(input1)  
  x = Conv1D(128, 12, activation='relu')(x)
  x = GRU(32)(x)
#   x = Conv1D(64, 7, activation='relu')(x)
#   x = Conv1D(128, 7, activation='relu')(x)
#   x = TimeDistributed(Dense(16, activation='relu'))(x)
#   x = Flatten()(x)
  x = Dense(256, activation='relu')(x)
  x = Dropout(0.5)(x)
  out = Dense(1, activation='sigmoid')(x)

  model = Model(inputs=input1, outputs=out)
  model.compile(optimizer=Adam(),
                loss='binary_crossentropy',
                metrics=['acc'])
  return model

In [21]:
################################################################################
# TODO#11                                                                      #
# Write code that call model.fit, or model.fit_generator if you have data      #
# generator, to train you models. Make sure you have validation_data as an     # 
# argument and use verbose=2 to generate one log line per epoch. Select your   #
# batch size carefully as it will affect your model's ability to converge and  #
# time needed for one epoch.                                                   #
#                                                                              #
# Hint: Read about callbacks_list argument on the documentation. You might     #
# find  ReduceLROnPlateau() and ModelCheckpoint() useful for your training     #
# process. Feel free to use any other callback function available.             #
################################################################################
print('start training best model')
my_best_model = get_my_best_model()
################################################################################
#                            WRITE YOUR CODE BELOW                             #
################################################################################
train_params = [(3, 512)]
for (epochs, batch_size) in train_params:
  print("train with {} epochs and {} batch size".format(epochs, batch_size))
  my_best_model.fit(x_train_char, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose,
                        #    callbacks=callbacks_list_feedforward_nn,
                           validation_data=(x_val_char, y_val))

start training best model
train with 3 epochs and 512 batch size
Epoch 1/3
32152/32152 [==============================] - 266s 8ms/step - loss: 0.0690 - acc: 0.9746 - val_loss: 0.0558 - val_acc: 0.9801
Epoch 2/3
32152/32152 [==============================] - 252s 8ms/step - loss: 0.0473 - acc: 0.9835 - val_loss: 0.0508 - val_acc: 0.9824
Epoch 3/3
32152/32152 [==============================] - 250s 8ms/step - loss: 0.0430 - acc: 0.9851 - val_loss: 0.0490 - val_acc: 0.9830


In [22]:
evaluate(x_test_char, y_test, my_best_model)

(0.9730329014585959, 0.9653136955869481, 0.980876556958826)

In [23]:
my_best_model = None
gc.collect()

34755